

this project tries to predict stars from imdb reviews. It is on a 10 star scale 6 and up is positive 5 or less is negative sentiment. I will aslo try to preduict the number of stars
step number1  is to load the  data. There are 50,000 total samples, 25,000 training samples and 25,000 testing samples. inside traing and testing  there are 12,500 positive reviews and 12,500 negative reviews.
I load the files and split the lines into a vector containing both the number of stars acheived and the and the word content of the review. The file is in LIBSVM format which means that the data is stored in a form of  "word" : "frequency". the word corresponds to an index in the imbd.vocab file, the frequency is number of times that specific word appears in the review.


In [13]:
#opening and reading the data
i = open('Trainlabeled.feat', 'r') 
j = open('labeledBow.feat', 'r')
training = [line.split(' ') for line in i.readlines()]
testing = [line.split(' ') for line in j.readlines()]

 Since  "word" is split from "frequency" with a colon I created a 2 dimensional array with the first index being the word number and the second the frequency This was then moved into a key value dictionary. i then put the dictionary and the review target was placed into another dictionary with two keys. The first key the number of starts the review gave to the movie. the second key si the word frequency pairs. these dictionarys are then put into an array to finish the preprocessing.



In [14]:
#data arrays
trainData = []
testData = []
# loop over all of the training features and testing features
for train_feature, test_feature in zip(training, testing):
  # split the frequency
  trainIN = [f.split(':') for f in train_feature[1:]] 
  testIN = [f.split(':') for f in test_feature[1:]]
  trainINDict = {}
  testInDict = {}
  # create key val pairs in the new dict

  for i in trainIN:
      trainINDict[int(i[0].strip('\n'))] = int(i[1].strip('\n'))

  for j in testIN:

    testInDict[int(j[0].strip('\n'))] = int(j[1].strip('\n'))
  # put the input and output data into the new dict
  trainDict2 = {'output': int(train_feature[0]), 'input': trainINDict}
  testDict2 = {'output': int(test_feature[0]), 'input': testInDict}
  # add dicts to the arrays
  trainData.append(trainDict2)
  testData.append(testDict2)
  # open the file
imdbEr = open('imdbEr.txt', 'r')

# read the value line by line
wordVals = [float(v.strip('\n')) for v in imdbEr.readlines()]

The first models to train will try to predict the exact number of stars as outputs.The sklearn models that are being implemented are LogisticRegression, MLPClassifier, and MLPRegressor. I will reduce each input down to a single value which corresponsds to the sum of all word polarities in the review. This will use just the word occurence, not frequency.

In [15]:
# calculuate polarity values for each review 
X_train_occurrence = []
X_test_occurrence = []

for i, test in zip(trainData, testData):
  sumTrain = 0
  sumTest = 0
  for word in i['input'].keys():
    sumTrain += wordVals[word]
  X_train_occurrence.append(sumTrain)
  for word in test['input'].keys():
    sumTest += wordVals[word]
  X_test_occurrence.append(sumTest)


  # collect targets into single array
y_train_multi = [train['output'] for train in trainData]
y_test_multi = [test['output'] for test in testData]

# reshape input arrays
import numpy as np
X_train_occurrence_np = np.array(X_train_occurrence).reshape(-1, 1)
X_test_occurrence_np = np.array(X_test_occurrence).reshape(-1, 1)

In [16]:
# create the models and test accuracy using occurence
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier, MLPRegressor

logistic_multi_occurrence = LogisticRegression()
logistic_multi_occurrence.fit(X_train_occurrence_np, y_train_multi)
logistic_multi_occurrence_accuracy = logistic_multi_occurrence.score(X_test_occurrence_np, y_test_multi)

mlp_multi_occurrence = MLPClassifier()
mlp_multi_occurrence.fit(X_train_occurrence_np, y_train_multi)
mlp_multi_occurrence_accuracy = mlp_multi_occurrence.score(X_test_occurrence_np, y_test_multi)

mlpr_multi_occurrence = MLPRegressor()
mlpr_multi_occurrence.fit(X_train_occurrence_np, y_train_multi)
mlpr_multi_occurrence_accuracy = mlpr_multi_occurrence.score(X_test_occurrence_np, y_test_multi)

print("Accuracy of multi class prediction using word occurrence")
print(f"Logistic Regression accuracy: {logistic_multi_occurrence_accuracy}")
print(f"MLP Classifier accuracy: {mlp_multi_occurrence_accuracy}")
print(f"MLP Regressor accuracy: {mlpr_multi_occurrence_accuracy}")

Accuracy of multi class prediction using word occurrence
Logistic Regression accuracy: 0.36648
MLP Classifier accuracy: 0.366
MLP Regressor accuracy: 0.5701906969305168


By employing multi-class output and word occurrence, it is apparent that the MLP Regressor provides the most accurate predictions with an accuracy rate of 57%. Considering that there were 10 potential classes and the model was correct over 50% of the time, this is not a terrible outcome. The other two models had almost identical accuracy rates, both being correct around one-third of the time. Given that a random guess would result in a 10% accuracy rate, anything above 10% is considered learning.

The next set of models will entail multi-class prediction utilizing word frequency as well. I plan to reduce the inputs once again and, this time, multiply the polarity value by the frequency of that word in the review. However, given that word occurrence is usually more crucial than frequency in the text, I do not expect this approach to dramatically enhance the predictions.

In [17]:
# calculuate polarity values for each review - this time incorporating word frequency
XTrainFreq = []
XTestFreq = []

for i, test in zip(trainData, testData):
  sumTrain = 0
  sumTest = 0
  for word in i['input'].keys():
    sumTrain += (wordVals[word] * i['input'][word])
  XTrainFreq.append(sumTrain)
  for word in test['input'].keys():
    sumTest += (wordVals[word] * test['input'][word])
  XTestFreq.append(sumTest)


  # reshape input arrays again
TrainFreqNP = np.array(XTrainFreq).reshape(-1, 1)
TestFreqNP = np.array(XTestFreq).reshape(-1, 1)

In [18]:
# create the models and test accuracy using frequency
logMultiFreq = LogisticRegression()
logMultiFreq.fit(TrainFreqNP, y_train_multi)
logMultiFreqAcc = logMultiFreq.score(TestFreqNP, y_test_multi)
mlpMultiFreq = MLPClassifier()
mlpMultiFreq.fit(TrainFreqNP, y_train_multi)
mlpMultiFreqAcc = mlpMultiFreq.score(TestFreqNP, y_test_multi)
mlprMultiFreq = MLPRegressor()
mlprMultiFreq.fit(TrainFreqNP, y_train_multi)
mlprMultiFreqAcc = mlprMultiFreq.score(TestFreqNP, y_test_multi)
print("Accuracy of word frequency")
print(f"Logistic Regression accuracy: {logMultiFreqAcc}")
print(f"MLP Classifier accuracy: {mlpMultiFreqAcc}")
print(f"MLP Regressor accuracy: {mlprMultiFreqAcc}")

Accuracy of word frequency
Logistic Regression accuracy: 0.35984
MLP Classifier accuracy: 0.3606
MLP Regressor accuracy: 0.5358764186343842


The outcomes are in line with expectations, showing similarity to the previous results. The MLP Regressor had the highest accuracy, while the other two models achieved comparable accuracies. Additionally, the results were somewhat inferior, providing evidence that word occurrence is more potent than word frequency. For the final models to be trained using this reduction method, I will be using binary classes (positive/negative) and the polarity values based solely on word occurrence, given its superior performance.

In [19]:
# collect targets into single array - using binary values
yTranBi = [1 if train['output'] > 5 else 0 for train in trainData]
yTestBi = [1 if test['output'] > 5 else 0 for test in testData]

In [20]:
# create the models and test accuracy using occurence and binary outputs
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier, MLPRegressor
logBiOCC = LogisticRegression()
logBiOCC.fit(X_train_occurrence_np, yTranBi)
logBiOccAcc = logBiOCC.score(X_test_occurrence_np, yTestBi)
mlpBIOcc = MLPClassifier()
mlpBIOcc.fit(X_train_occurrence_np, yTranBi)
mlpBiOccAcc = mlpBIOcc.score(X_test_occurrence_np, yTestBi)
mlprBiOcc = MLPRegressor()
mlprBiOcc.fit(X_train_occurrence_np, yTranBi)
mlprBiOccAcc = mlprBiOcc.score(X_test_occurrence_np, yTestBi)
print("Accuracy of  word occurrence")
print(f"Logistic Regression accuracy: {logBiOccAcc}")
print(f"MLP Classifier accuracy: {mlpBiOccAcc}")
print(f"MLP Regressor accuracy: {mlprBiOccAcc}")

Accuracy of  word occurrence
Logistic Regression accuracy: 0.8498
MLP Classifier accuracy: 0.85096
MLP Regressor accuracy: 0.5513295157570759


Changing to a binary output resulted in a significant improvement in prediction accuracy. Both the Logistic Regression and MLP Classifier now have an accuracy of around 85%. Interestingly, the MLP Regressor, which was the top predictor for multi-class output, is the poorest predictor for binary class outputs. This indicates that predicting binary outputs is easier and more successful. The optimal results have been achieved by combining binary outputs with using word occurrence instead of frequency.

# The Bag of Words

My plan for the next stage of the project involves using the bag of words technique, where the input comprises large feature vectors, with each vector dimension corresponding to a different word in the review dictionary. However, there are around 90,000 words and 50,000 samples, which would result in 4.5 billion numbers if each sample had an input vector with 90,000 dimensions. Considering each number is a double, this would amount to 36 billion bytes of data, which is too much memory. Therefore, I decided to use minibatch training, which involves training on small batches of the total input to use less memory while still training the model on all the data. I opted for a minibatch size of 1000 inputs, where each iteration of partial fitting sees only 1000 inputs, and the model will see every training sample over 25 iterations. I plan to train two models: the first one is an SGDClassifier, which behaves like logistic regression during partial fitting when the argument loss='log' is specified, and the second is an MLPClassifier, which performed the best when using the reduction method.

In [21]:
# define minibatch size and number of iterations
minibachSize = 1000
totalSize = np.ceil(len(trainData))
iterations = int(totalSize / minibachSize)
# create SGDClassifier
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log')

For the input vectors, I will solely rely on the word occurrence in the review, disregarding word frequency. The feature vector will contain the polarity value in that particular dimension only if the word appears in the review. If it does not appear, the feature vector will contain zero.

In [22]:
# train over number of iterations
for i in range(iterations): 
  xTrainMini = []
  # grab training targets
  yTrainMini = yTranBi[i * minibachSize:(i + 1) * minibachSize]
  xTestMini = []
  # grab testing targets
  yTestMini = yTestBi[i * minibachSize:(i + 1) * minibachSize]
  # generate feature vectors
  for train, test in zip(trainData[i * minibachSize:(i + 1) * minibachSize], testData[i * minibachSize:(i + 1) * minibachSize]):
    trainIN = train['input']
    trainVec = []
    testIN = train['input']
    testVec = []
    # look at all words and check if they exist in review words
    # if the word exists append the polarity value
    # if not, append 0
    for word in range(len(wordVals)):
      if word in trainIN:
        trainVec.append(wordVals[word])
      else:
        trainVec.append(0)
      if word in testIN:
        testVec.append(wordVals[word])
      else:
        testVec.append(0)
    # add vectors to the minibatch
    xTrainMini.append(trainVec)
    xTestMini.append(testVec)
  # test the fit after minibatches are created
  clf.partial_fit(xTrainMini, yTrainMini, [0, 1])
  valAcc = clf.score(xTestMini, yTestMini)
  trainAcc = clf.score(xTrainMini, yTrainMini)
  print(f"Iteration {i + 1}/{iterations}: training accuracy - {trainAcc}, validation accuracy - {valAcc}")

c:\Users\samta\OneDrive\Desktop\Racket\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


Iteration 1/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 2/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 3/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 4/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 5/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 6/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 7/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 8/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 9/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 10/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 11/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 12/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 13/25: training accuracy - 0.846, validation accuracy - 0.846
Iteration 14/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 15/25: training accuracy - 1.0, validation 

In [23]:
# create MLPClassifier - using 1000 neurons in hidden layer

from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(1000)

In [24]:
for i in range(iterations):
  xTrainMinibatch = []
  yTrainMini = yTranBi[i * minibachSize:(i + 1) * minibachSize]
  xTestMini = []
  yTestMinibatch = yTestBi[i * minibachSize:(i + 1) * minibachSize]
  for train, test in zip(trainData[i * minibachSize:(i + 1) * minibachSize], testData[i * minibachSize:(i + 1) * minibachSize]):
    trainIN = train['input']
    trainVec = []
    testIN = train['input']
    test_vector = []
    for word in range(len(wordVals)):
      if word in trainIN:
        trainVec.append(wordVals[word])
      else:
        trainVec.append(0)
      if word in testIN:
        test_vector.append(wordVals[word])
      else:
        test_vector.append(0)
    xTrainMinibatch.append(trainVec)
    xTestMini.append(test_vector)
  mlp.partial_fit(xTrainMinibatch, yTrainMini, [0, 1])
  valAcc = mlp.score(xTestMini, yTestMinibatch)
  trainAcc = mlp.score(xTrainMinibatch, yTrainMini)
  print(f"Iteration {i + 1}/{iterations}: training accuracy - {trainAcc}, validation accuracy - {valAcc}")


Iteration 1/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 2/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 3/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 4/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 5/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 6/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 7/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 8/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 9/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 10/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 11/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 12/25: training accuracy - 1.0, validation accuracy - 1.0
Iteration 13/25: training accuracy - 0.5, validation accuracy - 0.5
Iteration 14/25: training accuracy - 0.0, validation accuracy - 0.0
Iteration 15/25: training accuracy - 0.0, validation accu

In [25]:
import sklearn
train_data_shuffled = sklearn.utils.shuffle(trainData)
test_data_shuffled = sklearn.utils.shuffle(testData)
# collect shuffled targets
y_train_shuffled_binary = [1 if train['output'] > 5 else 0 for train in train_data_shuffled]
y_test_shuffled_binary = [1 if test['output'] > 5 else 0 for test in test_data_shuffled]

In [27]:
# logistic regression learning with shuffled inputs
clf_shuffled = SGDClassifier(loss='log')
for i in range(iterations): 
  xTrainMinibatch = []
  yTrainMini = y_train_shuffled_binary[i * minibachSize:(i + 1) * minibachSize]
  xTestMini = []
  yTestMinibatch = y_test_shuffled_binary[i * minibachSize:(i + 1) * minibachSize]
  for i, test in zip(train_data_shuffled[i * minibachSize:(i + 1) * minibachSize], test_data_shuffled[i * minibachSize:(i + 1) * minibachSize]):
    trainIN = i['input']
    trainVec = []
    testIN = i['input']
    testVec = []
    for word in range(len(wordVals)):
      if word in trainIN:
        trainVec.append(wordVals[word])
      else:
        trainVec.append(0)
      if word in testIN:
        testVec.append(wordVals[word])
      else:
        testVec.append(0)
    # add vectors to the minibatch
    xTrainMinibatch.append(trainVec)
    xTestMini.append(testVec)
  # test the fit after minibatches are created
  clf_shuffled.partial_fit(xTrainMinibatch, yTrainMini, [0, 1])
  valAcc = clf_shuffled.score(xTestMini, yTestMinibatch)
  trainAcc = clf_shuffled.score(xTrainMinibatch, yTrainMini)
  print(f"Iteration {i + 1}/{iterations}: training accuracy - {trainAcc}, validation accuracy - {valAcc}")

c:\Users\samta\OneDrive\Desktop\Racket\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


TypeError: unsupported operand type(s) for +: 'dict' and 'int'

In [ ]:
# mlp learning with shuffled inputs
mlp_shuffled = MLPClassifier(1000)

for i in range(iterations):
  xTrainMinibatch = []
  yTrainMini = y_train_shuffled_binary[i * minibachSize:(i + 1) * minibachSize]

  xTestMini = []
  yTestMinibatch = y_test_shuffled_binary[i * minibachSize:(i + 1) * minibachSize]
  
  for i, test in zip(train_data_shuffled[i * minibachSize:(i + 1) * minibachSize], test_data_shuffled[i * minibachSize:(i + 1) * minibachSize]):
    trainIN = i['input']
    trainVec = []
    testIN = i['input']
    testVec = []

    for word in range(len(wordVals)):
      if word in trainIN:
        trainVec.append(wordVals[word])
      else:
        trainVec.append(0)
      
      if word in testIN:
        testVec.append(wordVals[word])
      else:
        testVec.append(0)

    xTrainMinibatch.append(trainVec)
    xTestMini.append(testVec)

  mlp_shuffled.partial_fit(xTrainMinibatch, yTrainMini, [0, 1])
  valAcc = mlp_shuffled.score(xTestMini, yTestMinibatch)
  trainAcc = mlp_shuffled.score(xTrainMinibatch, yTrainMini)
  print(f"Iteration {i + 1}/{iterations}: training accuracy - {trainAcc}, validation accuracy - {valAcc}")

Iteration 1/25: training accuracy - 0.959, validation accuracy - 0.477
Iteration 2/25: training accuracy - 0.952, validation accuracy - 0.521
Iteration 3/25: training accuracy - 0.935, validation accuracy - 0.472
Iteration 4/25: training accuracy - 0.936, validation accuracy - 0.53
Iteration 5/25: training accuracy - 0.933, validation accuracy - 0.495
Iteration 6/25: training accuracy - 0.946, validation accuracy - 0.467
Iteration 7/25: training accuracy - 0.937, validation accuracy - 0.499
Iteration 8/25: training accuracy - 0.945, validation accuracy - 0.508
Iteration 9/25: training accuracy - 0.948, validation accuracy - 0.543
Iteration 10/25: training accuracy - 0.935, validation accuracy - 0.535
Iteration 11/25: training accuracy - 0.927, validation accuracy - 0.529
Iteration 12/25: training accuracy - 0.93, validation accuracy - 0.49
Iteration 13/25: training accuracy - 0.941, validation accuracy - 0.492
Iteration 14/25: training accuracy - 0.945, validation accuracy - 0.491
Iter

My accuraccy steadied around 50% for both the lmp and regression


The last thing I am going to try in this project is to do bag-of-words, but only on a certain number of the most common words.


The first step is to rank each word by totaling the number of times that word appeared in the reviews. I will only be looking at occurencde in each review of a word, not total number of times used. For words like "and", "the", and "but" this will reduce their total popularity.


Since only the training inputs are used to fit the model, I am only going to look at the words in the training data.

In [ ]:
def get_n_popular_words(n):
  words = {}
  for word in range(len(wordVals)):
    words[word] = 0
  # count the number of word occurences
  for train in trainData:
    inputs = train['input']
    for word in inputs:
      words[word] += 1
  sortedWords = sorted(words.items(), key=lambda item: item[1])
  nSortedWords = sortedWords[-n:]
  mostPopular = [p[0] for p in nSortedWords]
  return mostPopular

Now I can use feature vectors of any size, and hopefully fun the code faster to train more models. I am going to begin with the 1000 most popular words.

In [ ]:
def create_vector(input, most_popular):
  vector = []
  for word in most_popular:
    if word in input:
      vector.append(wordVals[word])
    else:
      vector.append(0)
  return vector
# logistic regression and mlp learning with 1000 most popular words
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
n = 1000 # number of popular words
# creating mlp
mlp1000MostPop = MLPClassifier(1000)
#creating logistic regression
lr1000MostPop = LogisticRegression(max_iter=200)
XTrain1000 = []
Xtest1000 = []
mostPop1000 = get_n_popular_words(n) # get most popular words
# generat vectors
for train, test in zip(trainData, testData):
  XTrain1000.append(create_vector(train['input'], mostPop1000))
  Xtest1000.append(create_vector(test['input'], mostPop1000))
lr1000MostPop.fit(XTrain1000, yTranBi)
lr1000MostPopAcc = lr1000MostPop.score(Xtest1000, yTestBi)
# training mlp
mlp1000MostPop.fit(XTrain1000, yTranBi)
mlp1000MostPopAcc = mlp1000MostPop.score(Xtest1000, yTestBi)
print("1000 most popular words")
print(f"Logistic Regression accuracy - {lr1000MostPopAcc}")
print(f"MLPClassifier accuracy - {mlp1000MostPopAcc}")

Using 1000 most popular words
Logistic Regression accuracy - 0.85864
MLPClassifier accuracy - 0.85444


Opting for the n most popular words leads to a significant improvement. By only utilizing the top 1000 most popular words, the accuracy of both the Logistic Regression and MLP Classifier models increased to approximately 85%. This is a noteworthy improvement, and these models are significantly more accurate than the prior bag-of-words models.

For the ultimate models, I plan to employ the same model types (Logistic Regression and MLP Classifier) while utilizing the top 5000 most popular words.

In [ ]:
# logistic regression and mlp learning with 5000 most popular words
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
n = 5000 # number of popular words
# creating mlp
mlp5000MostPop = MLPClassifier(1000)
#creating logistic regression
lr5000MostPop = LogisticRegression(max_iter=200)
xTrain5000 = []
xTest5000 = []
mostPop5000 = get_n_popular_words(n) # get most popular words
#  vectors
for train, test in zip(trainData, testData):
  xTrain5000.append(create_vector(train['input'], mostPop5000))
  xTest5000.append(create_vector(test['input'], mostPop5000))
lr5000MostPop.fit(xTrain5000, yTranBi)
lr5000MostPopAcc = lr5000MostPop.score(xTest5000, yTestBi)
# mlp
mlp5000MostPop.fit(xTrain5000, yTranBi)
mlp5000MostPopAcc = mlp5000MostPop.score(xTest5000, yTestBi)
print("5000 most popular words")
print(f"Logistic Regression accuracy - {lr5000MostPopAcc}")
print(f"MLPClassifier accuracy - {mlp5000MostPopAcc}")

Using 5000 most popular words
Logistic Regression accuracy - 0.87464
MLPClassifier accuracy - 0.87416


the models were quite sucessful. The initial bag-of-words models trained through minibatch iterations were still not practical for efficient training and there remains room for further improvement. The models that utilized vector reduction also had success but had problems. These models revealed that word occurrence is superior to word frequency. predicting sentiment overall is simpler than predicting the number of stars provided in a rating.

The models were able to improve accuracy by increasing the number of popular words from 1000 to 5000. The MLP model took longer to train than the logistic regression model, but both models achieved similar results.